<a href="https://colab.research.google.com/github/arissaharada/DIO_BairesDev_Machine_Learning/blob/main/DIO_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto de Transfer Learning em Python **
O projeto consiste em aplicar o método de Transfer Learning em uma rede de Deep Learning na linguagem Python no ambiente COLAB.  

Para exemplo, utilizaremos o seguinte projeto que realiza Transfer Learning com o Dataset do MNIST:
https://colab.research.google.com/github/kylemath/ml4a-guides/blob/master/notebooks/transfer-learning.ipynb

O dataset utilizado engloba duas classes: gatos e cachorros. Uma descrição da base de dados pode ser visualizada neste link: https://www.tensorflow.org/datasets/catalog/cats_vs_dogs.

Já o dataset para download pode ser acessado por meio deste outro link:

https://www.microsoft.com/en-us/download/details.aspx?id=54765.

In [1]:
!unzip -q kagglecatsanddogs_5340.zip -d /content/

In [ ]:
# Código completo: limpa imagens corrompidas salvando versões JPEG válidas e roda Transfer Learning
# Rode no Colab (Runtime > Change runtime type > GPU)

import os, random, shutil, math, time
from pathlib import Path
from PIL import Image, UnidentifiedImageError
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# ----------------------------
# CONFIGS
# ----------------------------
ORIG_ROOT = '/content/PetImages'       # pasta original que você descompactou
CLEAN_ROOT = '/content/PetImages_clean'  # pasta onde vamos salvar imagens "limpas" (será criada)
CLASSES = ['Cat', 'Dog']               # pastas esperadas em ORIG_ROOT
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123
AUTOTUNE = tf.data.AUTOTUNE
VALID_EXT = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff')

os.makedirs(CLEAN_ROOT, exist_ok=True)
for c in CLASSES:
    os.makedirs(os.path.join(CLEAN_ROOT, c), exist_ok=True)

# ----------------------------
# 1) varrer e "corrigir" imagens (re-salvar como JPEG convertidas para RGB)
# ----------------------------
print("1) Iniciando limpeza e copy/resave das imagens para:", CLEAN_ROOT)
start = time.time()
bad_files = []
copied = 0
skipped = 0

for cls in CLASSES:
    orig_dir = os.path.join(ORIG_ROOT, cls)
    clean_dir = os.path.join(CLEAN_ROOT, cls)
    if not os.path.isdir(orig_dir):
        raise FileNotFoundError(f"Pasta esperada não encontrada: {orig_dir}")
    files = os.listdir(orig_dir)
    for i, fname in enumerate(files):
        if not fname.lower().endswith(VALID_EXT):
            skipped += 1
            continue
        src = os.path.join(orig_dir, fname)
        try:
            with Image.open(src) as im:
                im = im.convert('RGB')             # força 3 canais
                # resave as JPEG into clean folder with unique name to avoid duplicates
                base_name = Path(fname).stem
                # evitar sobrescrever: acrescenta sufixo se já existir
                out_name = f"{base_name}.jpg"
                out_path = os.path.join(clean_dir, out_name)
                k = 1
                while os.path.exists(out_path):
                    out_name = f"{base_name}_{k}.jpg"
                    out_path = os.path.join(clean_dir, out_name)
                    k += 1
                im.save(out_path, format='JPEG', quality=95)
                copied += 1
        except (UnidentifiedImageError, OSError, ValueError, Exception) as e:
            # imagem corrompida ou problema — registra e pula
            bad_files.append(src)
        # logging periódico para não inundar o output
        if (i+1) % 2000 == 0:
            print(f"  [{cls}] processados: {i+1}/{len(files)}  (copiados até agora: {copied})")

elapsed = time.time() - start
print(f"Concluído. Imagens copiadas (limpas): {copied}. Arquivos pulados por extensão: {skipped}. Arquivos inválidos detectados: {len(bad_files)}")
if bad_files:
    print("Exemplo de arquivos inválidos (até 20):")
    for bf in bad_files[:20]:
        print(" ", bf)

# ----------------------------
# 2) construir listas de arquivos a partir do CLEAN_ROOT (agora todas JPEG)
# ----------------------------
filepaths = []
labels = []
for idx, cls in enumerate(CLASSES):
    p = os.path.join(CLEAN_ROOT, cls)
    for fname in os.listdir(p):
        if not fname.lower().endswith('.jpg'):
            continue
        filepaths.append(os.path.join(p, fname))
        labels.append(idx)

print("Total imagens válidas no CLEAN_ROOT:", len(filepaths))

# ----------------------------
# 3) embaralhar e split (70/15/15)
# ----------------------------
random.seed(SEED)
pairs = list(zip(filepaths, labels))
random.shuffle(pairs)
filepaths, labels = zip(*pairs)
filepaths, labels = list(filepaths), list(labels)

n = len(filepaths)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

train_files, train_labels = filepaths[:train_end], labels[:train_end]
val_files, val_labels = filepaths[train_end:val_end], labels[train_end:val_end]
test_files, test_labels = filepaths[val_end:], labels[val_end:]

print("Totals:", n, "train", len(train_files), "val", len(val_files), "test", len(test_files))

# ----------------------------
# 4) função de carregamento (usamos decode_jpeg já que re-salvamos como JPG)
# ----------------------------
def load_and_preprocess(path, label):
    image_data = tf.io.read_file(path)
    img = tf.io.decode_jpeg(image_data, channels=3)  # seguro: todas são JPG
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)
    return img, label

# ----------------------------
# 5) criação dos datasets (augmentation on-the-fly no train)
# ----------------------------
def make_dataset(files, labels, training=False):
    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    ds = ds.map(lambda p, l: load_and_preprocess(p, l), num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(2048, seed=SEED)
        aug = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.08),
            layers.RandomZoom(0.08),
        ])
        ds = ds.map(lambda x, y: (aug(x, training=True), y), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(train_files, train_labels, training=True)
val_ds = make_dataset(val_files, val_labels, training=False)
test_ds = make_dataset(test_files, test_labels, training=False)

# ----------------------------
# 6) construir modelo MobileNetV2
# ----------------------------
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))
base_model.trainable = False

inputs = layers.Input(shape=(*IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)
model.summary()

# ----------------------------
# 7) callbacks e treino da "cabeça"
# ----------------------------
callbacks = [
    keras.callbacks.ModelCheckpoint('/content/best_model.h5', monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=6,
    callbacks=callbacks
)

# ----------------------------
# 8) fine-tuning
# ----------------------------
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 50
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,                  # treino adicional (ajuste conforme VRAM/time)
    callbacks=callbacks
)

# ----------------------------
# 9) avaliar no test set e relatório
# ----------------------------
loss, acc, auc = model.evaluate(test_ds)
print("Test loss:", loss, "acc:", acc, "auc:", auc)

# predições completas
test_ds_full = tf.data.Dataset.from_tensor_slices((test_files, test_labels))
test_ds_full = test_ds_full.map(lambda p, l: load_and_preprocess(p, l))
test_ds_full = test_ds_full.batch(BATCH_SIZE)

y_probs = model.predict(test_ds_full, verbose=1).ravel()
y_pred = (y_probs >= 0.5).astype(int)
y_true = np.array(test_labels)

print(classification_report(y_true, y_pred, target_names=CLASSES))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))

# salvar modelo final
model.save('/content/final_model.h5')
print("Modelos salvos: /content/best_model.h5 e /content/final_model.h5")

# mostrar algumas previsões
import random
n_show = 8
idxs = random.sample(range(len(test_files)), n_show)
plt.figure(figsize=(16,6))
for i, idx in enumerate(idxs):
    img = Image.open(test_files[idx]).convert('RGB').resize(IMAGE_SIZE)
    plt.subplot(2,4,i+1)
    plt.imshow(img)
    prob = model.predict(np.expand_dims(preprocess_input(np.array(img).astype(np.float32)), axis=0))[0,0]
    label_pred = CLASSES[int(prob >= 0.5)]
    plt.title(f"{label_pred} ({prob:.2f})")
    plt.axis('off')
plt.show()


1) Iniciando limpeza e copy/resave das imagens para: /content/PetImages_clean
  [Cat] processados: 2000/12501  (copiados até agora: 2000)
  [Cat] processados: 4000/12501  (copiados até agora: 4000)
  [Cat] processados: 6000/12501  (copiados até agora: 5999)
  [Cat] processados: 8000/12501  (copiados até agora: 7999)
  [Cat] processados: 10000/12501  (copiados até agora: 9999)
  [Cat] processados: 12000/12501  (copiados até agora: 11998)
  [Dog] processados: 2000/12501  (copiados até agora: 14499)
  [Dog] processados: 4000/12501  (copiados até agora: 16499)
  [Dog] processados: 6000/12501  (copiados até agora: 18498)
  [Dog] processados: 8000/12501  (copiados até agora: 20498)
  [Dog] processados: 10000/12501  (copiados até agora: 22498)
  [Dog] processados: 12000/12501  (copiados até agora: 24497)
Concluído. Imagens copiadas (limpas): 24998. Arquivos pulados por extensão: 2. Arquivos inválidos detectados: 2
Exemplo de arquivos inválidos (até 20):
  /content/PetImages/Cat/666.jpg
  /con

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

...

547/547 ━━━━━━━━━━━━━━━━━━━━ 629s 1s/step - accuracy: 0.9741 - auc: 0.9959 - loss: 0.0800 - val_accuracy: 0.9869 - val_auc: 0.9989 - val_loss: 0.0476 - learning_rate: 1.0000e-04
Epoch 5/6
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 923ms/step - accuracy: 0.9761 - auc: 0.9971 - loss: 0.0686

547/547 ━━━━━━━━━━━━━━━━━━━━ 650s 1s/step - accuracy: 0.9761 - auc: 0.9971 - loss: 0.0686 - val_accuracy: 0.9877 - val_auc: 0.9991 - val_loss: 0.0434 - learning_rate: 1.0000e-04
Epoch 6/6
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 902ms/step - accuracy: 0.9755 - auc: 0.9974 - loss: 0.0651

547/547 ━━━━━━━━━━━━━━━━━━━━ 670s 1s/step - accuracy: 0.9755 - auc: 0.9974 - loss: 0.0651 - val_accuracy: 0.9875 - val_auc: 0.9991 - val_loss: 0.0409 - learning_rate: 1.0000e-04
Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9208 - auc: 0.9871 - loss: 0.1921

547/547 ━━━━━━━━━━━━━━━━━━━━ 860s 2s/step - accuracy: 0.9208 - auc: 0.9871 - loss: 0.1920 - val_accuracy: 0.9840 - val_auc: 0.9995 - val_loss: 0.0371 - learning_rate: 1.0000e-05
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9673 - auc: 0.9956 - loss: 0.0826

547/547 ━━━━━━━━━━━━━━━━━━━━ 862s 2s/step - accuracy: 0.9673 - auc: 0.9956 - loss: 0.0826 - val_accuracy: 0.9899 - val_auc: 0.9996 - val_loss: 0.0266 - learning_rate: 1.0000e-05
Epoch 3/10
249/547 ━━━━━━━━━━━━━━━━━━━━ 6:34 1s/step - accuracy: 0.9745 - auc: 0.9967 - loss: 0.0703